<a href="https://colab.research.google.com/github/thisishasan/prog_mod_b/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!wget -nc 'https://raw.githubusercontent.com/thisishasan/prog_mod_b/refs/heads/main/data.csv'

File ‘data.csv’ already there; not retrieving.



In [34]:
import pandas as pd

df = pd.read_csv("data.csv")

df = df[['Principio Attivo', 'Descrizione Gruppo', 'Denominazione e Confezione', 'Titolare AIC', 'AIC', 'Codice Gruppo Equivalenza']]
#df = df[df['AIC'] == 2129017]
#df = df.iloc[470:803]
#df = df.iloc[0:5]

# Display the first 5 rows of the DataFrame
print(df.head())

  Principio Attivo                     Descrizione Gruppo  \
0      Acamprosato  ACAMPROSATO 333MG 84 UNITA' USO ORALE   
1        Acarbosio    ACARBOSIO 100MG 40 UNITA' USO ORALE   
2        Acarbosio    ACARBOSIO 100MG 40 UNITA' USO ORALE   
3        Acarbosio    ACARBOSIO 100MG 40 UNITA' USO ORALE   
4        Acarbosio    ACARBOSIO 100MG 40 UNITA' USO ORALE   

            Denominazione e Confezione                    Titolare AIC  \
0  CAMPRAL*84 cpr riv gastrores 333 mg          BRUNO FARMACEUTICI SpA   
1               GLUCOBAY*40 cpr 100 mg                       BAYER SpA   
2              ACARPHAGE*40 cpr 100 mg          BRUNO FARMACEUTICI SpA   
3              ACARBOSIO*40 cpr 100 mg  TECNIMEDE SOC.TECNICO-MED.S.A.   
4              ACARBOSIO*40 cpr 100 mg                DOC GENERICI Srl   

        AIC Codice Gruppo Equivalenza  
0  34208013                       H0A  
1  26851016                       H1A  
2  38835144                       H1A  
3  39716182                 

In [35]:
df.isnull().sum()

,0
Principio Attivo,0
Descrizione Gruppo,0
Denominazione e Confezione,0
Titolare AIC,0
AIC,0
Codice Gruppo Equivalenza,0


In [36]:
df.duplicated().sum()

np.int64(0)

In [37]:
!pip install pymupdf

In [38]:
import requests
import fitz  # PyMuPDF
import re

rows_to_drop = []

# Iterate over each row and get structured data from the API
for index, row in df.iterrows():

    aic = row['AIC']

    # Build the API URL
    url = f"https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query={aic}&spellingCorrection=true&page=0"
    print(url)

    # Make the GET request
    response = requests.get(url)

    # Check for successful response
    if response.status_code == 200:
        json_data = response.json()  # Parse the response as JSON

        # Access the nested data field
        content = json_data.get('data', {}).get('content', [])
        print(content)
        if len(content) == 0:
          rows_to_drop.append(index)
          continue

        # Print each item in the content list
        for item in content:
            print(item)
            id = item.get('id')
            codice_atc = item.get('codiceAtc')[0]
            df.at[index, 'ATC'] = codice_atc
            descrizione_atc = item.get('descrizioneAtc')
            codice_sis = item.get('medicinale').get('codiceSis')
            aic6 = item.get('medicinale').get('aic6')
            leaflet_url = f"https://medicinali.aifa.gov.it/it/#/it/organizzazione/{codice_sis}/farmaci/{aic6}/stampati/FI"
            pdf_url = f"https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/organizzazione/{codice_sis}/farmaci/{aic6}/stampati?ts=RCP"
            df.at[index, 'URL'] = leaflet_url
            print(f"\n--- codiceAtc: {codice_atc} ---")
            print(f"\n--- descrizioneAtc: {descrizione_atc} ---")
            print(f"\n--- codiceSis: {codice_sis} ---")
            print(f"\n--- id: {id} ---")
            print(f"\n--- leafletUrl: {leaflet_url} ---")
            print(f"\n--- pdf_url: {pdf_url} ---")
            # Download the PDF
            response = requests.get(pdf_url)

            if response.status_code == 200:
                # Load and read the PDF
                pdf_path = 'rcp_downloaded.pdf'

                if response.status_code == 200:
                  with open("/content/rcp_downloaded.pdf", "wb") as f:
                      f.write(response.content)
                  print("PDF downloaded and saved as /content/rcp_downloaded.pdf")
                else:
                  print(f"Failed to download PDF. Status code: {response.status_code}")

                doc = fitz.open(pdf_path)

                # Combine all page text into one string
                full_text = ""
                for page in doc:
                    full_text += page.get_text()
                doc.close()

                # Normalize the text: fix words broken across lines
                text = re.sub(r'([a-zA-Z])\n([a-zA-Z])', r'\1 \2', full_text)  # fix broken words
                text = re.sub(r'\n+', '\n', text)  # collapse extra newlines

                # Define section headings (number + title)
                sections = [
                    ("4.1", "Indicazioni terapeutiche"),
                    ("4.2", "Posologia e modo di somministrazione"),
                    ("4.3", "Controindicazioni"),
                    ("4.4", "Avvertenze speciali e precauzioni di impiego"),
                    ("4.5", "Interazioni con altri medicinali"),
                    ("4.6", "Fertilità, gravidanza e allattamento"),
                    ("4.7", "Effetti sulla capacità di guidare"),
                    ("4.8", "Effetti indesiderati"),
                    ("4.9", "Sovradosaggio"),
                    ("6.2", "Incompatibilità")
                ]

                # Create a regex pattern that is tolerant to newlines/spaces
                pattern = '|'.join([
                    fr"{re.escape(num)}[\s\n\.]*{re.escape(title.split()[0])}"  # match start of title
                    for num, title in sections
                ])
                matches = list(re.finditer(pattern, text, flags=re.IGNORECASE))

                # Extract section content
                extracted = {}
                for i in range(len(matches)):
                    matched_header = matches[i].group().strip()
                    start = matches[i].end()
                    end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

                    # Identify correct section name
                    section_key = next((f"{num} {title}" for num, title in sections if num in matched_header), matched_header)
                    content = text[start:end].strip()
                    extracted[section_key] = content

                # Save each section in a separate variable
                for title, content in extracted.items():
                    section_name = title.replace(" ", "_").replace(".", "")  # Create a valid variable name
                    globals()[section_name] = content  # Dynamically assign the content to a variable
                    df.at[index, title] = content

                    # Print the results
                    print(f"\n{'='*80}\n{title}\n{'='*80}\n{content[:1000]}...\n")

    else:
        print(f"Request failed for AIC {aic}: Status Code {response.status_code}")

df = df.drop(rows_to_drop).reset_index(drop=True)

Streaming output truncated to the last 5000 lines.
https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query=41195645&spellingCorrection=true&page=0
[]
https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query=41966045&spellingCorrection=true&page=0
[]
https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query=43057088&spellingCorrection=true&page=0
[]
https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query=43695170&spellingCorrection=true&page=0
[]
https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query=43696259&spellingCorrection=true&page=0
[]
https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query=43697174&spellingCorrection=true&page=0
[]
https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query=32944148&spellingCorrection=true&page=0
[]
https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query=40433221&spellingCorrection=true&page=0
[]
https://api.a

In [39]:
df.head()

,Principio Attivo,Descrizione Gruppo,Denominazione e Confezione,Titolare AIC,AIC,Codice Gruppo Equivalenza,ATC,URL,4.1 Indicazioni terapeutiche,4.2 Posologia e modo di somministrazione,4.3 Controindicazioni,4.4 Avvertenze speciali e precauzioni di impiego,4.5 Interazioni con altri medicinali,"4.6 Fertilità, gravidanza e allattamento",4.7 Effetti sulla capacità di guidare,4.8 Effetti indesiderati,4.9 Sovradosaggio,6.2 Incompatibilità
0,Acido folico,ACIDO FOLICO 5MG 20 UNITA' USO ORALE,FOLINA*20 cps molli 5 mg,TEOFARMA Srl,2309045,H7B,G03HA01,https://medicinali.aifa.gov.it/it/#/it/organiz...,terapeutiche \n \nTrattamento antiandrogeno ne...,e modo di somministrazione \n \n \nPosologia \...,• \nIpersensibilità al principio attivo o ad u...,speciali e precauzioni di impiego \n \nFegato ...,con altri medicinali ed altre forme di interaz...,gravidanza e allattamento \n \nSe assunto per ...,sulla capacità di guidare veicoli e sull’uso d...,indesiderati \n \nI più frequenti effetti inde...,Studi di tossicità acuta a seguito di somminis...,"In assenza di studi di compatibilità, questo m..."
1,Acido folico,ACIDO FOLICO 5MG 28 UNITA' USO ORALE,FOLINA*28 cps molli 5 mg,TEOFARMA Srl,2309060,H7C,V03AH01,https://medicinali.aifa.gov.it/it/#/it/organiz...,terapeutiche \n \nProglicem è indicato per il ...,e modo di somministrazione \n \nPosologia \nLo...,Ipersensibilità al principio attivo o ad uno q...,speciali e precauzioni di impiego \n \nI pazie...,con altri medicinali ed altre forme di interaz...,gravidanza e allattamento \n \nGravidanza \nPr...,sulla capacità di guidare veicoli e sull’uso d...,indesiderati \n \nComuni e gravi: ritenzione i...,La comparsa di iperglicemia o glicosuria può r...,Non pertinente. \n \n6.3 \nPeriodo di validità...
2,Acido folico,ACIDO FOLICO 15MG 5 UNITA' USO PARENTERALE,FOLINA*5 fiale IM 15 mg 2 ml,TEOFARMA Srl,2309033,H7D,G03HA01,https://medicinali.aifa.gov.it/it/#/it/organiz...,terapeutiche \n \nTrattamento antiandrogeno ne...,e modo di somministrazione \n \n \nPosologia \...,• \nIpersensibilità al principio attivo o ad u...,speciali e precauzioni di impiego \n \nFegato ...,con altri medicinali ed altre forme di interaz...,gravidanza e allattamento \n \nSe assunto per ...,sulla capacità di guidare veicoli e sull’uso d...,indesiderati \n \nI più frequenti effetti inde...,Studi di tossicità acuta a seguito di somminis...,"In assenza di studi di compatibilità, questo m..."
3,Acido folico,ACIDO FOLICO 5MG 60 UNITA' USO ORALE,FOLINA*60 cps molli 5 mg,TEOFARMA Srl,2309058,H7E,R02AX03,https://medicinali.aifa.gov.it/it/#/it/organiz...,terapeutiche \nBENZIDAMINA ACRAF 3 mg/ml spray...,e modo di somministrazione \n \nAdulti: 2-4 sp...,Agenzia Italiana del Farmaco Documento reso di...,speciali e precauzioni di impiego \n \nIn un n...,con altri medicinali ed altre forme di interaz...,gravidanza e allattamento \n \nNon ci sono dat...,sulla capacità di guidare veicoli e sull’uso d...,indesiderati \n \nLa tabella sottostante ripor...,In caso di sovradosaggio svuotare lo stomaco p...,La soluzione di benzidamina non presenta incom...
4,Acido folico,ACIDO FOLICO 5MG 120 UNITA' USO ORALE,FOLINA*120 cps molli 5 mg,TEOFARMA Srl,2309072,JHI,G03HA01,https://medicinali.aifa.gov.it/it/#/it/organiz...,terapeutiche \n \nTrattamento antiandrogeno ne...,e modo di somministrazione \n \n \nPosologia \...,• \nIpersensibilità al principio attivo o ad u...,speciali e precauzioni di impiego \n \nFegato ...,con altri medicinali ed altre forme di interaz...,gravidanza e allattamento \n \nSe assunto per ...,sulla capacità di guidare veicoli e sull’uso d...,indesiderati \n \nI più frequenti effetti inde...,Studi di tossicità acuta a seguito di somminis...,"In assenza di studi di compatibilità, questo m..."


In [40]:
df.to_csv('updated_data.csv', index=False)
print("\nUpdated CSV saved as updated_data.csv")


Updated CSV saved as updated_data.csv
